In [ ]:
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray
import glob
import os

# main 

In [ ]:
path = 'org/'
files = os.listdir(path)
files_dir = [f for f in files if os.path.isdir(os.path.join(path, f))]

In [ ]:
for fd in files_dir:
    fp = glob.glob(path + fd + '\\*.shp')[0]
    gdf = gpd.read_file(fp)
    
    ind = gdf["G02_001"].values
    val = gdf["G02_014"].values # Total annual rainfall
    
    # size 1st order Chiki-Mesh
    x = np.full((80,80), int(65000), dtype='uint16') # y, x
    
    for dp, v in zip(ind, val):
        # yindex
        iy = 10 * int(dp[4]) + int(dp[6])
        # yindex
        ix = 10 * int(dp[5]) + int(dp[7])
        x[iy,ix] = int(v)
    
    dp = ind[0]
    lonp = float(100) + float(dp[2:4])  + float(0.5*45/3600)
    latp = float(dp[:2]) * float(40/60) + float(0.5*30/3600)
    
    lon = lonp + np.arange(80)*45/3600
    lat = latp + np.arange(80)*30/3600
    
    # y is North to South
    ds = xr.Dataset({
                     'value': (['y','x'], x[::-1])                 
                    }
                    , coords={
                                'x': lon
                              , 'y': lat[::-1]
                             }
                        , attrs={'crs':'+init=epsg:' + str(4326)}
                    )
    
    dsp = ds['value']
    d = dsp.rio.write_crs('epsg:4326', inplace=True)
    out = dsp.rio.to_raster(f'geotiff\\{dp[:4]}.tif') #, compress='zstd')

# make VRT

In [ ]:
from osgeo import gdal 

# gdal 2.1以降
my_vrt = gdal.BuildVRT('geotiff/output.vrt', glob.glob( 'geotiff/*.tif'), VRTNodata=65000, srcNodata=65000)
my_vrt = None

#  VRT to Raster

In [ ]:
ds = xr.open_rasterio('geotiff/output.vrt')

d = ds.rio.write_crs('epsg:4326', inplace=True)
out = ds.rio.to_raster('geotiff\\AverageAnnualRainfall2010.tif', compress='zstd')

# graphing

In [ ]:
import hvplot.xarray
import geoviews as gv
gv.extension('bokeh')

In [ ]:
ds = xr.open_rasterio('geotiff/AverageAnnualRainfall2010.tif')

In [ ]:
dsn = xr.where(ds == 65000, np.nan, ds)
dsn.attrs = ds.attrs

In [ ]:
geomap = gv.WMTS('https://mt1.google.com/vt/lyrs=y&x={X}&y={Y}&z={Z}', name="GoogleMapsImagery")
fig = dsn.isel(band=0).hvplot.image(rasterize=False, geo=True, project=True
                             , dynamic=False, cmap='jet', colorbar=True, clim=(0,40000), alpha=0.5).options(clipping_colors={'NaN': 'transparent'})
g = geomap*fig
go = g.options(title='Average annual rainfall 2010 [x10mm]', width=500, height=500)

In [ ]:
d = hvplot.save(go,'map.html')